In [3]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors


In [4]:
products = pd.read_csv("products.csv")
purchase = pd.read_csv("Purchase_His.csv")

In [6]:
final_dataset = purchase.pivot(index='StockCode',columns='CustomerID',values='Purchase')

In [7]:
final_dataset.fillna(0,inplace=True)

In [8]:
no_user_purchased = purchase.groupby('StockCode')['Purchase'].agg('count')
no_products_purchased = purchase.groupby('CustomerID')['Purchase'].agg('count')

In [9]:
final_dataset = final_dataset.loc[no_user_purchased[no_user_purchased > 10].index,:]

In [11]:
final_dataset=final_dataset.loc[:,no_products_purchased[no_products_purchased > 50].index]

In [12]:
csr_data = csr_matrix(final_dataset.values)
final_dataset.reset_index(inplace=True)

In [13]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(csr_data)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [14]:
def get_product_recommendation(product_name):
    n_products_to_reccomend = 5
    product_list = products[products['Description'].str.contains(product_name)]  
    if len(product_list):        
        product_idx= product_list.iloc[0]['StockCode']
        product_idx = final_dataset[final_dataset['StockCode'] == product_idx].index[0]
        distances , indices = knn.kneighbors(csr_data[product_idx],n_neighbors=n_products_to_reccomend+1)    
        rec_product_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        recommend_frame = []
        for val in rec_product_indices:
            product_idx = final_dataset.iloc[val[0]]['StockCode']
            idx = products[products['StockCode'] == product_idx].index
            recommend_frame.append({'Description':products.iloc[idx]['Description'].values[0],'StockCode':products.iloc[idx]['StockCode'].values[0],'Distance':val[1]})
        df = pd.DataFrame(recommend_frame,index=range(1,n_products_to_reccomend+1))
        return df
    else:
        return "No Products found. Please check your input"

In [15]:
get_product_recommendation('RED WOOLLY HOTTIE WHITE HEART')

,Description,StockCode,Distance
1,KNITTED UNION FLAG HOT WATER BOTTLE,84029G,0.664128
2,CHOCOLATE HOT WATER BOTTLE,22112,0.654891
3,SCOTTIE DOG HOT WATER BOTTLE,22111,0.646132
4,RETROSPOT HEART HOT WATER BOTTLE,21485,0.645688
5,WHITE SKULL HOT WATER BOTTLE,21479,0.620196


In [16]:
get_product_recommendation('GARLAND, VINTAGE BELLS')

,Description,StockCode,Distance
1,RABBIT NIGHT LIGHT,23084,0.790690
2,VINTAGE JINGLE BELLS WREATH,23492,0.785896
3,HEART OF WICKER SMALL,22469,0.761799
4,BELL HEART DECORATION,23103,0.735304
5,VINTAGE JINGLE BELLS HEART,23491,0.727991


In [17]:
get_product_recommendation('TUSCAN VILLA BIRD TABLE')

,Description,StockCode,Distance
1,LARGE PARLOUR PICTURE FRAME,23457,0.855662
2,SET OF 4 DIAMOND NAPKIN RINGS,84813,0.811018
3,LOVE GARLAND PAINTED ZINC,22341,0.803040
4,WALL MIRROR RECTANGLE DIAMANTE PINK,22166,0.776393
5,TUSCAN VILLA DOVECOTE,23466,0.716527
